In [ ]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from torch import nn,optim
import torch

import algo
import attack
import scripts

import os
import matplotlib.pyplot as plt
import json

rand_seed=24
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)

In [8]:
# purchase
# load data with different seeds
data_seed = {}
for rand_seed in [1,3,13,24,42]:
    data_seed[rand_seed] =  {}
    
    data_seed[rand_seed]['x_target_train'] = np.load('data/rs'+str(rand_seed)+'_x_target_train.npy')
    data_seed[rand_seed]['y_target_train'] = np.load('data/rs'+str(rand_seed)+'_y_target_train.npy')
    data_seed[rand_seed]['x_target_test'] = np.load('data/rs'+str(rand_seed)+'_x_target_test.npy')
    data_seed[rand_seed]['y_target_test'] = np.load('data/rs'+str(rand_seed)+'_y_target_test.npy')
    data_seed[rand_seed]['n_classes'] = len(np.unique(data_seed[rand_seed]['y_target_train']))
    data_seed[rand_seed]['X_train_size'] = data_seed[rand_seed]['x_target_train'].shape[0]
    data_seed[rand_seed]['X_test_size ']= data_seed[rand_seed]['x_target_test'].shape[0]

def set_the_seed_and_data(seed):
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    random.seed(rand_seed)
    
    return data_seed[seed]['x_target_train'], data_seed[rand_seed]['y_target_train'], data_seed[rand_seed]['x_target_test'], data_seed[rand_seed]['y_target_test']

In [9]:
# attack models
from torch import nn

class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

path = 'mia'
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            ams[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            ams[file] = torch.load(r+'/'+file)      

# MI on centralized 

In [ ]:
path = 'tm'
tms_params = {}
tms = {}
for r,d,f in os.walk(path):
    for file in f:
        if "target_model_params.json" in file:
            with open(path+'/'+file) as json_file:
                tms_params[file.replace('_params.json', '')] = json.load(json_file)
        if "target_model.npy" in file:
            tms[file.replace('.npy', '')] = np.load(path+'/'+file)
            
df = pd.DataFrame.from_dict(tms_params, orient='index')
df.shape            

In [ ]:
# attack every centralized target model
for file in tms_params:
    
    if 'attack_acc_mean' in tms_params[file]:
        continue
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_lr')])
    x_target_train, y_target_train, x_target_test, y_target_test = set_the_seed_and_data(rand_seed)
    
    #attack
    target_model = algo.LogisticRegression_DPSGD()
    target_model.theta = tms[file]
    params = tms_params[file]
    scripts.set_model_params(target_model, params)
    attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
    
    params.update(attack_dict)
    if 'attack_acc' in params:
        params.pop('attack_acc')
        params.pop('attack_pre')
        params.pop('attack_rec')
    #write a new parameters file with attack results
    with open('tm/'+file+'_params.json', 'w') as file:
        json.dump(params, file)
        

# Membership inference Federated Learning


In [186]:
path = 'fl/'
params = {}
results = {}
models = {}
for r,d,f in os.walk(path):
    for file in f:
        if "params.json" in file:
            with open(r+'/'+file) as json_file:
                params[r] = json.load(json_file)
        elif "results.csv" in file:
            results[r] = pd.read_csv(r+'/'+file)
            it = []
            client = []
            model_filenames = []
            if 'Unnamed: 0' in results[r].keys():
                for k in results[r]['Unnamed: 0']:
                    it.append(k[k.find('i')+1:k.find('_')])
                    client.append(k[k.find('_')+1:])
                    model_filenames.append(k+'.npy')
                results[r]['it'] = it
                results[r]['client'] = client
                results[r]['file_name'] = model_filenames
                results[r].pop('Unnamed: 0')
        elif '.npy' in file:
            if r not in models:
                models[r] = {}
            models[r][file] = np.load(r+'/'+file)



In [187]:
selected_files = results.keys()
len(selected_files)

187

In [188]:
# attack every federated local/global model
for file in selected_files:
    
    if 'attack_acc_mean' in results[file].keys():
        continue
        
    # set random seed and load the data    
    rand_seed=int(file[file.find('rs')+2:file.find('_ncl')])
    x_target_train, y_target_train, x_target_test, y_target_test = set_the_seed_and_data(rand_seed)
    #set number of client for following split of the training data between clients
    number_of_clients = len(results[file]['client'].unique())-1
    data_per_client = int(x_target_train.shape[0]/number_of_clients)

    #attack
    attack_results = {}
    for tm in results[file]['file_name']:
        target_model = algo.LogisticRegression_DPSGD()
        target_model.theta = models[file][tm]
        tm_params = params[file]
        scripts.set_model_params(target_model, tm_params)
        if 'g' in tm:
            target_model.x = x_target_train
            target_model.y = y_target_train
        else:
            i = int(tm[tm.find('_c')+2:tm.find('.npy')])
            target_model.x = x_target_train[i*data_per_client:(i+1)*data_per_client]
            target_model.y = y_target_train[i*data_per_client:(i+1)*data_per_client]
        attack_dict = attack.test_mi_attack(ams, target_model, target_model.x, target_model.y, x_target_test, y_target_test)
        attack_results[tm] = attack_dict
    attack_df = pd.DataFrame.from_dict(attack_results, orient='index')
    result_df = results[file].set_index('file_name')
    new_df = pd.merge(result_df, attack_df, left_index=True, right_index=True)
    #save attack results in old results file
    new_df.to_csv(file+'/results.csv')
    results[file] = new_df

In [179]:
new_df

,train_acc,test_acc,it,client,attack_acc_mean,attack_acc_std,attack_pre_mean,attack_pre_std,attack_rec_mean,attack_rec_std
file_name,,,,,,,,,,
i0_c0.npy,0.9550,0.4167,0,c0,0.709217,0.006318,0.638744,0.005079,0.963367,0.005884
i0_c1.npy,0.9610,0.4121,0,c1,0.720217,0.008519,0.648104,0.007755,0.964200,0.006840
i0_g.npy,0.8843,0.5015,0,g,0.657917,0.005122,0.604279,0.003784,0.915233,0.008484
i1_c0.npy,0.9842,0.4536,1,c0,0.691983,0.007415,0.622023,0.006301,0.979167,0.005920
i1_c1.npy,0.9826,0.4455,1,c1,0.697467,0.009590,0.626945,0.008409,0.976133,0.008496
i1_g.npy,0.9420,0.4948,1,g,0.662758,0.006824,0.602527,0.005475,0.956983,0.006433
i2_c0.npy,0.9926,0.4559,2,c0,0.691617,0.008404,0.621064,0.006985,0.983633,0.007149
i2_c1.npy,0.9934,0.4442,2,c1,0.695717,0.010407,0.624954,0.009078,0.980000,0.008927
i2_g.npy,0.9673,0.4880,2,g,0.670942,0.008228,0.606915,0.006610,0.971033,0.006244


In [175]:
# path = 'fl/'
# params = {}
# results = {}
# models = {}
# for r,d,f in os.walk(path):
#     if 'outDPlocalTrue' not in r:
#         continue
# #     for file in f:
# #         if '_outDP.npy' not in file and 'before_DP.npy' not in file and '.npy' in file:
# #             os.rename(r+'/'+file, r+'/'+file.replace('.npy','')+'_before_DP.npy')
#     for file in f:
#         if '.npy' in file:
#             print(r+'/'+file)
# #             os.remove(r+'/'+file)
# #             if r not in models:
# #                 models[r] = {}
# #             models[r][file] = np.load(r+'/'+file)



fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i0_c1_before_DP.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i2_c0.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i2_c1.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i0_c0.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i3_c0_before_DP.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i4_c1_before_DP.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i0_c1.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i1_c1_before_DP.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i4_c1.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i4_c0.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i2_c0_before_DP.npy
fl/rs42_ncl2_fiter5_lr0.001_iter10_reg0.0001_outDPlocalTrue_eps0.5/i3_c1.npy
fl/rs42_ncl2_fiter5_lr0.00